In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Laguna,US,2020-09-15 17:18:45,38.42,-121.42,69.01,68,1,3.94
1,1,Vila Velha,BR,2020-09-15 17:18:45,-20.33,-40.29,80.60,69,20,18.34
2,2,Northam,GB,2020-09-15 17:18:46,51.03,-4.22,64.00,93,84,4.00
3,3,Tuktoyaktuk,CA,2020-09-15 17:18:46,69.45,-133.04,30.20,92,90,14.99
4,4,Port Lincoln,AU,2020-09-15 17:18:46,-34.73,135.87,55.44,80,100,10.38


In [6]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [8]:
# # Get the maximum temperature.
# max_temp = city_data_df["Max Temp"]
# temps = []
# for temp in max_temp:
#     temps.append(max(temp, 0))

In [12]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
# Heatmap of percent wind
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [17]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Vila Velha,BR,2020-09-15 17:18:45,-20.33,-40.29,80.60,69,20,18.34
5,5,Karratha,AU,2020-09-15 17:14:15,-20.74,116.85,75.40,58,80,8.66
6,6,Hambantota,LK,2020-09-15 17:18:47,6.12,81.12,80.60,81,48,16.28
12,12,Butaritari,KI,2020-09-15 17:18:48,3.07,172.79,83.41,75,36,20.65
18,18,Mayumba,GA,2020-09-15 17:18:51,-3.43,10.66,75.36,86,91,11.79
21,21,Srivardhan,IN,2020-09-15 17:18:52,18.03,73.02,79.50,85,100,2.75
29,29,Reyes,BO,2020-09-15 17:18:55,-14.32,-67.38,82.40,61,75,9.17
30,30,Camacha,PT,2020-09-15 17:18:55,33.08,-16.33,75.20,64,20,8.05
33,33,Ankara,TR,2020-09-15 17:16:26,39.92,32.85,78.80,24,0,6.93
34,34,Victoria,HK,2020-09-15 17:18:56,22.29,114.16,84.00,83,86,8.01


In [18]:
preferred_cities_df.count()

City_ID       192
City          192
Country       192
Date          192
Lat           192
Lng           192
Max Temp      192
Humidity      192
Cloudiness    192
Wind Speed    192
dtype: int64

In [19]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Vila Velha,BR,80.60,-20.33,-40.29,
5,Karratha,AU,75.40,-20.74,116.85,
6,Hambantota,LK,80.60,6.12,81.12,
12,Butaritari,KI,83.41,3.07,172.79,
18,Mayumba,GA,75.36,-3.43,10.66,
21,Srivardhan,IN,79.50,18.03,73.02,
29,Reyes,BO,82.40,-14.32,-67.38,
30,Camacha,PT,75.20,33.08,-16.33,
33,Ankara,TR,78.80,39.92,32.85,
34,Victoria,HK,84.00,22.29,114.16,


In [22]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

# Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name.
    try: 
      hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Vila Velha,BR,80.60,-20.33,-40.29,Hotel Vitória Palace
5,Karratha,AU,75.40,-20.74,116.85,ibis Styles Karratha
6,Hambantota,LK,80.60,6.12,81.12,Bungalow 63
12,Butaritari,KI,83.41,3.07,172.79,Isles Sunset Lodge
18,Mayumba,GA,75.36,-3.43,10.66,LIKWALE LODGE
21,Srivardhan,IN,79.50,18.03,73.02,Savli Resort/Hotel - Shrivardhan
29,Reyes,BO,82.40,-14.32,-67.38,
30,Camacha,PT,75.20,33.08,-16.33,Hotel Porto Santo & Spa
33,Ankara,TR,78.80,39.92,32.85,Sheraton Ankara Hotel & Convention Center
34,Victoria,HK,84.00,22.29,114.16,Mini Hotel Central


In [23]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [26]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [27]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90
